# Fine-tuning MentalBERT on GoEmotions (Ekman subset)

This notebook loads the Ekman-labelled slice of GoEmotions, adds a classification head to MentalBERT, fine-tunes the model, and exports a saved checkpoint plus an inference helper.

In [1]:
%pip install --upgrade -q transformers datasets accelerate evaluate scikit-learn pandas

Note: you may need to restart the kernel to use updated packages.


In [13]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torch import nn
from transformers import (AutoConfig, AutoTokenizer, BertModel, BertPreTrainedModel,
                          DataCollatorWithPadding, Trainer, TrainingArguments)
from transformers.modeling_outputs import SequenceClassifierOutput
from IPython.display import display

In [15]:
SEED = 42
VAL_SPLIT = 0.1
MAX_LENGTH = 128
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 32
MODEL_NAME = "mental/mental-bert-base-uncased"
LABEL_COLUMNS = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
DATA_DIR = Path('dataset/go_emotions_ekman')
CSV_PATH = DATA_DIR / 'ekman_1.csv'
OUTPUT_DIR = Path('mentalbert-goemotions-output')
FINAL_MODEL_DIR = Path('mentalbert-goemotions-ekman-model')

np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FINAL_MODEL_DIR.mkdir(parents=True, exist_ok=True)

In [16]:
raw_df = pd.read_csv(CSV_PATH)
raw_df['labels'] = raw_df[LABEL_COLUMNS].apply(lambda row: row.astype(int).tolist(), axis=1)
filtered_df = raw_df[raw_df[LABEL_COLUMNS].sum(axis=1) > 0].reset_index(drop=True)

train_df, val_df = train_test_split(filtered_df, test_size=VAL_SPLIT, random_state=SEED, shuffle=True)
train_df = train_df[['text', 'labels']].reset_index(drop=True)
val_df = val_df[['text', 'labels']].reset_index(drop=True)

print(f'Training samples: {len(train_df)}')
print(f'Validation samples: {len(val_df)}')
display(train_df.head())

Training samples: 61983
Validation samples: 6888


,text,labels
0,He probably enjoyed it so much with his frat b...,"[0, 0, 0, 1, 0, 0, 0]"
1,"lol, not a bad idea I could always ~~use more ...","[0, 0, 0, 1, 0, 0, 0]"
2,"By all means, go for it. We'll all be much hap...","[0, 0, 0, 1, 0, 0, 0]"
3,take it easy,"[0, 0, 0, 0, 0, 0, 1]"
4,It would do us ALL a great deal of good.,"[0, 0, 0, 1, 0, 0, 0]"


In [17]:
train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
val_dataset = Dataset.from_pandas(val_df, preserve_index=False)
datasets = DatasetDict({'train': train_dataset, 'validation': val_dataset})
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 61983
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 6888
    })
})

In [18]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(batch):
    tokens = tokenizer(batch['text'], truncation=True, max_length=MAX_LENGTH)
    tokens['labels'] = batch['labels']
    return tokens

tokenized_datasets = datasets.map(preprocess, batched=True, remove_columns=['text'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets

Map: 100%|██████████| 6888/6888 [00:00<00:00, 81409.29 examples/s]


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 61983
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6888
    })
})

In [19]:
id2label = {i: label for i, label in enumerate(LABEL_COLUMNS)}
label2id = {label: i for i, label in id2label.items()}

config = AutoConfig.from_pretrained(
    MODEL_NAME,
    num_labels=len(LABEL_COLUMNS),
    id2label=id2label,
    label2id=label2id,
    problem_type='multi_label_classification',
)

class MentalBertForMultiLabelClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        dropout_prob = (
            config.classifier_dropout
            if getattr(config, 'classifier_dropout', None) is not None
            else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.post_init()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,
        )
        pooled_output = (
            outputs.pooler_output
            if outputs.pooler_output is not None
            else outputs.last_hidden_state[:, 0]
        )
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            labels = labels.float()
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

model = MentalBertForMultiLabelClassification.from_pretrained(MODEL_NAME, config=config)

Some weights of MentalBertForMultiLabelClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probabilities = 1 / (1 + np.exp(-logits))
    predictions = (probabilities >= 0.5).astype(int)
    return {
        'micro_f1': f1_score(labels, predictions, average='micro', zero_division=0),
        'macro_f1': f1_score(labels, predictions, average='macro', zero_division=0),
        'subset_accuracy': accuracy_score(labels, predictions),
    }

In [22]:
training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    learning_rate=1e-5,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='micro_f1',
    greater_is_better=True,
    logging_steps=100,
    report_to='none',
    seed=SEED,
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/tmp/ipykernel_5504/1492615138.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Micro F1,Macro F1,Subset Accuracy
1,0.237300,0.289593,0.556695,0.445617,0.465592
2,0.216700,0.309459,0.556840,0.453946,0.466173
3,0.218800,0.319866,0.553662,0.449206,0.468496


TrainOutput(global_step=11622, training_loss=0.21526452654344738, metrics={'train_runtime': 284.9299, 'train_samples_per_second': 652.613, 'train_steps_per_second': 40.789, 'total_flos': 3272559826089000.0, 'train_loss': 0.21526452654344738, 'epoch': 3.0})

In [23]:
trainer.evaluate()

{'eval_loss': 0.30945852398872375,
 'eval_micro_f1': 0.5568404974907266,
 'eval_macro_f1': 0.453945762179,
 'eval_subset_accuracy': 0.4661730545876887,
 'eval_runtime': 1.6231,
 'eval_samples_per_second': 4243.636,
 'eval_steps_per_second': 133.076,
 'epoch': 3.0}

In [24]:
trainer.save_model(str(FINAL_MODEL_DIR))
tokenizer.save_pretrained(str(FINAL_MODEL_DIR))
with open(FINAL_MODEL_DIR / 'label_mapping.json', 'w') as fp:
    json.dump({'id2label': id2label, 'label2id': label2id}, fp, indent=2)
print(f'Model and tokenizer saved to {FINAL_MODEL_DIR.resolve()}')

Model and tokenizer saved to /home/chahar/mentalBert/project_B/mentalbert-goemotions-ekman-model


In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inference_tokenizer = AutoTokenizer.from_pretrained(str(FINAL_MODEL_DIR))
inference_model = MentalBertForMultiLabelClassification.from_pretrained(str(FINAL_MODEL_DIR)).to(device)
inference_model.eval()

def predict_emotions(texts, threshold=0.5):
    if isinstance(texts, str):
        texts = [texts]
    encodings = inference_tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=MAX_LENGTH,
        return_tensors='pt',
    )
    encodings = {k: v.to(device) for k, v in encodings.items()}
    with torch.no_grad():
        outputs = inference_model(**encodings)
        probs = torch.sigmoid(outputs.logits).cpu().numpy()
    results = []
    for text, prob in zip(texts, probs):
        active_labels = [LABEL_COLUMNS[i] for i, score in enumerate(prob) if score >= threshold]
        results.append({
            'text': text,
            'emotions': active_labels,
            'scores': {LABEL_COLUMNS[i]: float(prob[i]) for i in range(len(LABEL_COLUMNS))},
        })
    return results

predict_emotions([
    'I am thrilled with the good news!',
    'I feel anxious about tomorrow.',
])

[{'text': 'I am thrilled with the good news!',
  'emotions': ['joy'],
  'scores': {'anger': 0.004906625486910343,
   'disgust': 0.0014321267371997237,
   'fear': 0.0018056827830150723,
   'joy': 0.9821141362190247,
   'sadness': 0.003681192174553871,
   'surprise': 0.014119678176939487,
   'neutral': 0.011779725551605225}},
 {'text': 'I feel anxious about tomorrow.',
  'emotions': ['fear'],
  'scores': {'anger': 0.054138533771038055,
   'disgust': 0.010227530263364315,
   'fear': 0.7941871285438538,
   'joy': 0.053097207099199295,
   'sadness': 0.07931454479694366,
   'surprise': 0.054462216794490814,
   'neutral': 0.09180615097284317}}]